# Training notebook
Used to replicate training procedures. Based on the code of this repo: https://github.com/hrsht-13/Breast-Cancer-Detection
You will need the phantom images in the corresponding folder

In [ ]:
# Imports
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from keras_preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
import albumentations as A
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
import cv2
import os
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout,AveragePooling2D
from tensorflow.keras.applications.resnet import ResNet50
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3,DenseNet201,EfficientNetB7, MobileNetV2,Xception,VGG16,NASNetMobile
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from keras.models import Sequential
from keras.regularizers import *
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [ ]:
def preprocess(image):
    kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    im = cv2.filter2D(image, -1, kernel)
    #out2 = skimage.exposure.rescale_intensity(im, in_range=(150,200), out_range=(0,255))
    out1=(cv2.normalize(im, (224,224),0, 255, cv2.NORM_MINMAX))
    return out1

In [ ]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [ ]:
# Loading the dataset and splitting it into train and validation
train=pd.read_csv("content/phantom_images/Training_set.csv")
X_train, X_valid = train_test_split(train, test_size=0.1,stratify=train["stratlabel"],shuffle=True, random_state = 42)

In [ ]:
# Train generator
datagen=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess)
train_generator=datagen.flow_from_dataframe(
dataframe=X_train,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

In [ ]:
# Validation generator
valid_datagen=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess)
valid_generator=valid_datagen.flow_from_dataframe( 
dataframe=X_valid,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

## Experiment1 1:
Only last 3 neurons

In [ ]:
checkpoint_path = "experiment1_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
for layer in conv_base.layers:
        layer.trainable = False
model.layers[-1].trainable = True
model.layers[-3].trainable = False
model.layers[-5].trainable = False
model.layers[-7].trainable = False

In [ ]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

## Experiment 2
8 and 3 neuron layers

In [ ]:
checkpoint_path = "experiment2_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
for layer in conv_base.layers:
        layer.trainable = False
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = False
model.layers[-7].trainable = False

In [ ]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

## Experimento 3
All dense layers

In [ ]:
checkpoint_path = "experiment3_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
for layer in conv_base.layers:
        layer.trainable = False
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True
model.layers[-7].trainable = False

In [ ]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

# Experiment 4
Copy to mammography training

In [ ]:
checkpoint_path = "experiment4_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [ ]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

# Experiment 5
ResNet less defreeze

In [ ]:
checkpoint_path = "experiment5_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = ResNet50(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:3]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)

model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [ ]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

# Experiment 6
ResNet

In [ ]:
checkpoint_path = "experiment6_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = ResNet50(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)

model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [ ]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

# Experiment 7
Inception

In [ ]:
checkpoint_path = "experiment7_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = InceptionV3(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)

model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [ ]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

# Experiment 8
Inception more lr

In [ ]:
checkpoint_path = "experiment7_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = InceptionV3(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/phantoms_training_experiment7_seeded.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [ ]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

# Experiment 9
Change learning rate

In [ ]:
checkpoint_path = "experiment4_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [ ]:
model.fit(
    train_generator,
    #steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=70,
    validation_data=valid_generator,
    #validation_steps=STEP_SIZE_VALID,
    #callbacks=[my_callbacks]
)

# Experiment 10
Data augmentation

In [ ]:
datagen_aug=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess,vertical_flip=True)
train_generator_aug=datagen_aug.flow_from_dataframe(
dataframe=X_train,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

In [ ]:
checkpoint_path = "experiment4_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [ ]:
model.fit(
    train_generator_aug,
    epochs=70,
    validation_data=valid_generator,
)

# Experiment 11
More weight decay to the best model

In [ ]:
checkpoint_path = "experiment4_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [ ]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

# Experiment 12
More augmentation

In [ ]:
datagen_aug=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess,
                               vertical_flip=True, 
                               horizontal_flip = True,
                              zoom_range = 0.2)
train_generator_aug=datagen_aug.flow_from_dataframe(
dataframe=X_train,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

In [ ]:
checkpoint_path = "training_0/exp12_best.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, monitor = "val_accuracy",save_best_only = True, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [ ]:
model.fit(
    train_generator_aug,
    epochs=70,
    validation_data=valid_generator,
    callbacks=[my_callbacks]
)